<a href="https://colab.research.google.com/github/shivavsrivastava/Algorithms/blob/main/Course4_W1_Johnsons_BellmanFord_Dijkstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All Pair Shortest Paths



In [ ]:
import numpy as np
import random
import urllib3
import math
from collections import defaultdict
import time
import heapq
from heapq import heappush, heappop, heapify
import sys
from itertools import count

import networkx as nx
import matplotlib.pyplot as plt

## Bellman-Ford Algorithm

The Bellman-Ford algorithm solves the single-source shortest-path problem. While slower than Dijkstra's algorithm, it accommodates negative edge lengths and is also better suited for distributed implementations.

Runtime O(mn)

Bellman Ford Algorithm using Dynamic Programming with:

► Negative edge cost

► Space Optimization

► Stopping Early

► Negative Cycle Detection

► Reconstruction Algorithm in case where negative cycle is not detected


In [ ]:
def Bellman_Ford_Algorithm_dynamic(graph, A, B, nodes, i_start_vert):
    row, col = A.shape
    #print("rows = {}, columns = {}".format(row, col))
    negative_cycle = False
    early_stopped = False
    for i in range(1, col+1):
        if all(A[0] == A[1]):  # stopping early
            early_stopped = True
            break
        A[0] = A[1]
        B[0] = B[1]
        for v in range(col):
            case_sum_subproblems = []
            case_1 = A[0][v]
            case_1_b = B[0][v]
            for pre in graph.predecessors(nodes[v]):
                w = nodes.index(pre)
                #print("i: {}, iw: {}, w: {}, v: {}, sw: {}, wv: {} ".format(i, w,nodes[w], nodes[v], A[0][w] , graph.edges[nodes[w], nodes[v]]['weight']))
                case_sum_subproblems.append([A[0][w] + graph.edges[nodes[w], nodes[v]]['weight'], w])
                #print(case_sum_subproblems)
            if len(case_sum_subproblems) != 0:
                case_2 = min(case_sum_subproblems)
                case_2_b = case_2[1]
                #print("case_1: {} case_2: {}".format(case_1, case_2))
                if case_1 <= case_2[0]:
                    A[1][v] = case_1
                    B[1][v] = case_1_b
                else:
                    A[1][v] = case_2[0]
                    B[1][v] = case_2_b
                #print(A)
                #print(B)
    if any(A[0] != A[1]) and (early_stopped == False) :  # Negative Cycle Detection
        negative_cycle = True
    return negative_cycle


def reconstruction_algorithm(graph, A, B, nodes, i_start_vert):
    print("Bellman Ford Algorithm Reconstructing the shortest paths: ")
    for v in range(len(nodes)):
        path_node = v
        path = []
        while path_node != i_start_vert:
            path.append(nodes[path_node])
            path_node = B[1][path_node]
        path.append(nodes[i_start_vert])
        #print("({}, {}) : {}".format(nodes[i_start_vert], nodes[v], path[::-1]))


def Bellman_Ford_Algorithm_optimize(graph, start_vert):
    nodes = list(graph.nodes)
    #A = np.full((len(nodes)-1, len(nodes)), None)
    A = np.full((2, len(nodes)), None)   # space optimization
    B = np.full((2, len(nodes)), None)   # Reconstruction Algorithm
    i_start_vert = nodes.index(start_vert)
    A[1] = float('inf')
    A[1][i_start_vert] = 0
    B[1][i_start_vert] = i_start_vert
    negative_cycle = Bellman_Ford_Algorithm_dynamic(graph, A, B, nodes, i_start_vert)
    if negative_cycle == False:
        #print("A : \n{}".format(A))
        shortest_paths = A[-1]
        print("Bellman Ford Algorithm - Shortest path distance:")
        for idx in range(len(shortest_paths)):
            print("({}, {}): {}".format(start_vert, nodes[idx], shortest_paths[idx]))

        reconstruction_algorithm(graph, A, B, nodes, i_start_vert)
        return [list(shortest_paths), nodes]

    else:
        print("Error: Negative Cycle detected in graph. Bellman-Ford Algorithm does not allow negative cycle.")
        return False



In [ ]:
# Test 1: Negative Edge and Reconstruction algorithm

DG = nx.DiGraph()
for i,j in zip(['s', 'v', 'w', 'x', 't'],[(0, 5), (5, 10), (10, 10), (7.5, 0), (15, 5)]):
    DG.add_node(i, pos=j)
DG.nodes.data()
DG.add_weighted_edges_from([('s', 'v', 0), ('v', 'w', 2), ('w', 't', 2), ('t', 'w', 2), ('s', 'x', 4), ('v', 'x', -3), ('x', 't', -5)])
pos = nx.get_node_attributes(DG, 'pos')
weights = nx.get_edge_attributes(DG, 'weight')
nx.draw_networkx_edge_labels(DG, pos, edge_labels = weights)
nx.draw_networkx_labels(DG,pos=pos)
nx.draw(DG, pos, with_labels = True)
start_vert = 's'
Bellman_Ford_Algorithm_optimize(DG, start_vert)

In [ ]:
## Test2: Negative Cycle
DG = nx.DiGraph()
for i,j in zip(['s', 'v', 'w', 'x', 't'],[(0, 5), (5, 10), (10, 10), (7.5, 0), (15, 5)]):
    DG.add_node(i, pos=j)
DG.nodes.data()
DG.add_weighted_edges_from([('s', 'v', 0), ('v', 'w', 2), ('w', 't', 2), ('t', 'w', 2), ('s', 'x', -4), ('x', 's', -4), ('v', 'x', -3), ('x', 't', -5)])
pos = nx.get_node_attributes(DG, 'pos')
weights = nx.get_edge_attributes(DG, 'weight')
nx.draw_networkx_edge_labels(DG, pos, edge_labels = weights)
nx.draw_networkx_labels(DG,pos=pos)
nx.draw(DG, pos, with_labels = True)
start_vert = 's'
Bellman_Ford_Algorithm_optimize(DG, start_vert)

In [ ]:
def dijkstra_heap(DG, source):
    #Heap implementation which does the following
    #
    # 1. For vertices in X, find all edges originating from them to all vertices not in X
    # 2. Keep track of minimum value of len(w) + lwv
    # 3. Return w, v and lwv
    X = [source]
    minHeap = []
    heapq.heappush(minHeap, [0, source])
    all_nodes = list(DG.nodes)
    for node in all_nodes:
        DG.nodes[node]['shortest_dist'] = float('inf')
    while len(minHeap) != 0:
        w = heapq.heappop(minHeap)
        X.append(w[1])
        DG.nodes[w[1]]['shortest_dist'] = w[0]
        for edge in list(DG.edges):
            if (edge[0] == w[1]) and (edge[1] not in X):  # node that has just been popped should be the tail
                dji_greedy = w[0] + DG.edges[edge[0], edge[1]]['weight'] #djikstra's greedy criterion
                doublenp = np.array(minHeap)
                singlenp = []
                if len(doublenp) == 0:
                    heapq.heappush(minHeap, [dji_greedy, edge[1]])
                    continue
                else:
                    singlenp = doublenp[:,1]
                if edge[1] not in singlenp:
                    heapq.heappush(minHeap, [dji_greedy, edge[1]])
                else:
                    dest_idx = np.where(singlenp == edge[1])
                    if dji_greedy < int(doublenp[dest_idx][0][0]):
                        minHeap[dest_idx[0][0]] = minHeap[0]
                        heapq.heapify(minHeap)
                        heapq.heappop(minHeap)
                        heapq.heappush(minHeap, [dji_greedy, edge[1]])
    shortest_path_dict = {}
    for node in all_nodes:
        shortest_path_dict[node] = DG.nodes[node]['shortest_dist']
    return shortest_path_dict

In [ ]:
def dijkstra_heap2(DG, source):
    #Heap implementation which does the following
    #
    # 1. For vertices in X, find all edges originating from them to all vertices not in X
    # 2. Keep track of minimum value of len(w) + lwv
    # 3. Return w, v and lwv
    X = [source]
    minHeap = []
    heapq.heappush(minHeap, [0, source])
    all_nodes = list(DG.nodes)
    for node in all_nodes:
        DG.nodes[node]['shortest_dist'] = float('inf')
    while len(minHeap) != 0:
        w = heapq.heappop(minHeap)
        # If the node already in X, then ignore
        if w[1] in X and w[1]!=source:
          continue
        X.append(w[1])
        DG.nodes[w[1]]['shortest_dist'] = w[0]
        for edge in list(DG.edges):
            if (edge[0] == w[1]) and (edge[1] not in X):  # node that has just been popped should be the tail
                dji_greedy = w[0] + DG.edges[edge[0], edge[1]]['weight'] #djikstra's greedy criterion
                heapq.heappush(minHeap, [dji_greedy, edge[1]])
    shortest_path_dict = {}
    for node in all_nodes:
        shortest_path_dict[node] = DG.nodes[node]['shortest_dist']
    return shortest_path_dict

In [ ]:
def Johnsons_Algorithm(graph):
    nodes = list(graph.nodes)
    graph_prime = graph.copy(as_view=False)
    graph_prime.add_node('Johnsons')
    j_edges_add = []
    for node in nodes:
        j_edges_add.append(('Johnsons', node, 0))
    graph_prime.add_weighted_edges_from(j_edges_add)
    j_start_vert = 'Johnsons'
    bellman_Pv = Bellman_Ford_Algorithm_optimize(graph_prime, j_start_vert)
    if bellman_Pv == False:
        return False
    graph_prime_dij = graph.copy(as_view=False)
    for e in graph_prime_dij.edges:
        Pu = bellman_Pv[0][bellman_Pv[1].index(e[0])]
        Pv = bellman_Pv[0][bellman_Pv[1].index(e[1])]
        graph_prime_dij[e[0]][e[1]]['weight'] = graph[e[0]][e[1]]['weight'] + Pu - Pv
    print("\nRunning Dijkstra on all paths")
    apsp = {}
    for node in nodes:
        shortest_dict = dijkstra_heap2(graph_prime_dij, node)
        apsp[node] = shortest_dict
        #print("Dijkstra {} : {}".format(node, shortest_dict))
    print("\nJohnson\'s Algorithm : Shortest path format head : [tail: shortest path values] : \n")
    shortest_shortest_apsp = float('inf')
    for source, dest in apsp.items():
        for d in dest:
            Pu = bellman_Pv[0][bellman_Pv[1].index(source)]
            Pv = bellman_Pv[0][bellman_Pv[1].index(d)]
            dest[d] -= (Pu - Pv)
            if shortest_shortest_apsp > dest[d]:
                shortest_shortest_apsp = dest[d]
        print("{}  :  {}".format(source, dest))
    return shortest_shortest_apsp


In [ ]:
# Test 3: Bellman-Ford and Johnsons
DG = nx.DiGraph()
for i,j in zip(['s', 'v', 'w', 'x', 't'],[(0, 5), (5, 10), (10, 10), (7.5, 0), (15, 5)]):
    DG.add_node(i, pos=j)
DG.nodes.data()
DG.add_weighted_edges_from([('s', 'v', 0), ('v', 'w', 2), ('w', 't', 2), ('t', 'w', 2), ('s', 'x', 4), ('v', 'x', -3), ('x', 't', -5)])
pos = nx.get_node_attributes(DG, 'pos')
weights = nx.get_edge_attributes(DG, 'weight')
nx.draw_networkx_edge_labels(DG, pos, edge_labels = weights)
nx.draw_networkx_labels(DG,pos=pos)
nx.draw(DG, pos, with_labels = True)
Johnsons_Algorithm(DG)

## Programming Assignment



In this assignment you will implement one or more algorithms for the all-pairs shortest-path problem.  Here are data files describing three graphs:

**g1.txt**

**g2.txt**

**g3.txt**



The first line indicates the number of vertices and edges, respectively.  Each subsequent line describes an edge (the first two numbers are its tail and head, respectively) and its length (the third number).  NOTE: some of the edge lengths are negative.  NOTE: These graphs may or may not have negative-cost cycles.

Your task is to compute the "shortest shortest path".  Precisely, you must first identify which, if any, of the three graphs have no negative cycles.  For each such graph, you should compute all-pairs shortest paths and remember the smallest one (i.e., compute
min
⁡
𝑢
,
𝑣
∈
𝑉
𝑑
(
𝑢
,
𝑣
)
min
u,v∈V
​
 d(u,v), where
𝑑
(
𝑢
,
𝑣
)
d(u,v) denotes the shortest-path distance from
𝑢
u to
𝑣
v).

If each of the three graphs has a negative-cost cycle, then enter "NULL" in the box below.  If exactly one graph has no negative-cost cycles, then enter the length of its shortest shortest path in the box below.  If two or more of the graphs have no negative-cost cycles, then enter the smallest of the lengths of their shortest shortest paths in the box below.

In [ ]:
def testcase_Johnson_Algorithm(url):
    test_graph = nx.DiGraph()
    http = urllib3.PoolManager()
    r1 = http.request('GET', url)
    IntegerMatrixStringJoin = r1.data.decode('utf8').split('\n')
    IntegerMatrixStringJoin.remove('')
    IntegerMatrixStringJoin.remove(IntegerMatrixStringJoin[0])
    edges = []
    for i in IntegerMatrixStringJoin:
        node_edges = i.split(' ')
        source = int(node_edges[0])
        dest = int(node_edges[1])
        length = int(node_edges[2])
        edges.append((source, dest, length))
    test_graph.add_weighted_edges_from(edges)
    print("Graph created")
    output = Johnsons_Algorithm(test_graph)
    return output


In [ ]:
G1_shortest_path = testcase_Johnson_Algorithm("https://d3c33hcgiwev3.cloudfront.net/_6ff856efca965e8774eb18584754fd65_g1.txt?Expires=1716163200&Signature=jYFeKGlFQU0C97jT0jsDCs-7ARgln-gGfBq1Fh6HRfhqJfh0XmulAHOsk6~d3j0yJt6sb6wC-7CpwFOtFKIdmncZNtuY8YYWt~U1gGsUpPHHDNRNLL5fxP4zar5eHnxht~zZ9gF~iBKEWKC2gknHvR26u0g5ZpBx9kzrh9~kUZA_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")
print(G1_shortest_path)

Ans: Negative cycle detected!

The Johnson Algorithm took a long time : 7 minutes

In [ ]:
G2_shortest_path = testcase_Johnson_Algorithm("https://d3c33hcgiwev3.cloudfront.net/_6ff856efca965e8774eb18584754fd65_g2.txt?Expires=1716163200&Signature=jYGtQy-VcqMic6OctmbIRbaGHAzTDYCvbsEbqgBq8U7yVUF5s-wfCrUY0-KNxGlDEqrwAEX92ypDNcm~gvHOm60dssAXAvocr1f31u8y0osDYEtJ23BfURvITxA5whOCjTa3JV61jQHdpHIPdpL326YCMkkUDfbO3Ji49zyICSw_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")
print(G2_shortest_path)

Ans: Negative cycle detected!

The Johnson Algorithm took a long time : 7 minutes 30 seconds

In [ ]:
G3_shortest_path = testcase_Johnson_Algorithm("https://d3c33hcgiwev3.cloudfront.net/_6ff856efca965e8774eb18584754fd65_g3.txt?Expires=1716163200&Signature=g2g2unVy85a54I6fcVTksw5SUcI~gHJ2p6MEWcoelfYC6nCLtP5~CffHn4pq5UPpXNE6Jlc1aROP7lYiQA3dwOkSdS24PJ1Yj5WUnczwEvyu4BPcnyNYbAd8MbZcIvqa7hVUgPQ7BFKaQXW~IYvz46Y37JkJVxIdSVyYdBpWXcY_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")
print(G3_shortest_path)

Ans: -19

algorithm took 1 hr 30 sec